In [1]:
import numpy as np
import tensorflow as tf
import cv2
import os
from random import shuffle
from tqdm import tqdm


/home/aditya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Reading labels

In [2]:
import pandas as pd
y=pd.read_csv('labels.csv')

### Loading Images and assigning them classes and resizing them

In [ ]:
img_width = 120
img_height = 120
images=[]
classes=[]
#load training images
for i in range(120):
    for x in os.listdir("dog_train"):
        if i+1<=len(os.listdir('dog_train/'+x)) :
            img = cv2.imread('dog_train/'+x+'/'+os.listdir(os.path.join("dog_train",x))[i])
            resize_image = cv2.resize(img, (img_width,img_height))
            images.append(resize_image)
            classes.append(x)

### Creating one hot for classes

In [4]:
y = np.array(pd.get_dummies(classes,dtype="float32"))

## Image Augmentation
### This function will take a list of images and for each image there is 50% chance it will get passed to placeholder without any modification or it will get augmented in any of the 8 augmentations:  Flip (updown or left/right), Dilation, Blur, Rotation, Shear,Translation, RGB.

In [5]:
def augment(image):
    Aug_operation= ['flip_horz','flip_up','blur','dilation','rgb_','rotate','translate','sheer']
    img = []
    out=[]
    aug_img = []
    img_width = (image[0]).shape[0]
    img_height = (image[0]).shape[1]
    n_op = len(Aug_operation)
    for j in range(len(image)): 
        apply_aug =np.random.randint(0,2)
        if apply_aug == 0:
            aug_img = image[j]
        else:
            rand_int= np.random.randint(0,n_op)
            rows,cols,ch = image[0].shape 
            if rand_int%n_op == 0:
                aug_img = (cv2.flip(image[j],1))
            elif rand_int%n_op ==1:        
                aug_img=(cv2.flip(image[j],0))
            elif rand_int%n_op ==2:
                aug_img=(cv2.blur(image[j],(3,3)))
            elif rand_int%n_op ==3:
                kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2))
                aug_img=(cv2.dilate(image[j],kernel_ellipse,iterations=1))
            elif rand_int%n_op ==4:
                aug_img=cv2.cvtColor(image[j],cv2.COLOR_BGR2RGB)
            elif rand_int%n_op ==5:
                angle= np.random.uniform(-1,1)*90
                Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1)
                aug_img=(cv2.warpAffine(image[j],Rot_M,(cols,rows)))
            elif rand_int%n_op ==6:
                x_shift = np.random.uniform(-1,1)*img_width/8
                y_shift = np.random.uniform(-1,1)*img_height/8
                Trans_M = np.float32([[1,0,x_shift],[0,1,y_shift]])
                aug_img=(cv2.warpAffine(image[j],Trans_M,(cols,rows)))         
            elif rand_int%n_op ==7:           
                pts1 = np.float32([[5,5],[20,5],[5,20]])
                pts2 = np.float32([[10,10],[25,17],[8,20]])
                M = cv2.getAffineTransform(pts1,pts2)
                aug_img = cv2.warpAffine(image[j],M,(cols,rows))
        out.append(aug_img)
    return out

### Weight Initialization

In [6]:
def init_weight(shape):
    w=tf.Variable(tf.truncated_normal(shape,stddev=0.1))
    return w

### Bias Initialization

In [7]:
def init_biases(shape):
    b=tf.Variable(tf.constant(0.1,shape=shape))
    return b

### Convolution

In [8]:
def conv2d(X1,w):
    return tf.nn.conv2d(X1,w,strides=[1,1,1,1],padding='SAME')

### Max Pooling

In [9]:
def max_pool(X1):
    return tf.nn.max_pool(X1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

### Applies Convolution -> Add Bias -> ReLU Activation 

In [10]:
def convolution_layer(X1,shape):
    w=init_weight(shape)
    b=init_biases([shape[3]])
    return tf.nn.relu(conv2d(X1,w)+b)

### Function for Fully Connected Layer

In [11]:
def fully_connected_layer(X1,size):
    feature=int(X1.get_shape()[1])
    w=init_weight([feature,size])
    b=init_biases([size])
    return tf.matmul(X1,w)+b

### Placeholder for Input and label (Image Size=[120,120])

In [12]:
X1=tf.placeholder(tf.float32,shape=[None,120,120,3])
y_true=tf.placeholder(tf.float32,shape=[None,120])

### Convolution(3,3,channel(3->16)) -> Max Pooling(2,2) -> Convolution(3,3,channel(16->32)) -> Max Pooling(2,2)

In [13]:
conv_1=convolution_layer(X1,shape=[3,3,3,16])
max_pool1=max_pool(conv_1)
conv_2=convolution_layer(max_pool1,shape=[3,3,16,32])
max_pool2=max_pool(conv_2)

In [14]:
# lrn1=tf.nn.local_response_normalization(
#     max_pool2,
#     depth_radius=5,
#     bias=1,
#     alpha=1,
#     beta=0.5,
#     name=None
# )

### Convolution(3,3,channel(32->64)) -> Max Pooling(2,2)

In [15]:
conv_3=convolution_layer(max_pool2,shape=[3,3,32,64])
max_pool3=max_pool(conv_3)
max_pool3

<tf.Tensor 'MaxPool_2:0' shape=(?, 15, 15, 64) dtype=float32>

### Convolution(3,3,channel(64->128))

In [16]:
conv_4=convolution_layer(max_pool3,shape=[3,3,64,128])
conv_4

<tf.Tensor 'Relu_3:0' shape=(?, 15, 15, 128) dtype=float32>

In [17]:
#max_pool4=max_pool(conv_4)

In [18]:
# lrn2=tf.nn.local_response_normalization(
#     max_pool4,
#     depth_radius=5,
#     bias=1,
#     alpha=1,
#     beta=0.5,
#     name=None
# )

In [19]:
#conv_5=convolution_layer(lrn2,shape=[3,3,64,128])
#max_pool5=max_pool(conv_5)
#conv_6=convolution_layer(max_pool5,shape=[3,3,128,256])
#max_pool6=max_pool(conv_6)
#hold_prob = tf.placeholder(tf.float32)
#full_one_dropout = tf.nn.dropout(max_pool6,keep_prob=hold_prob)

### Flatten

In [20]:
depth_flat=tf.reshape(conv_4,[-1,128*225])

### Fully Connected Layer(1024) -> Dropout -> Prediction

In [21]:
fcn1=tf.nn.relu(fully_connected_layer(depth_flat,1024))
hold_prob = tf.placeholder(tf.float32)
full_2_dropout = tf.nn.dropout(fcn1,keep_prob=hold_prob)
#fcn2=tf.nn.relu(fully_connected_layer(full_2_dropout,512))
#full_3_dropout = tf.nn.dropout(fcn2,keep_prob=hold_prob)
y_pred=(fully_connected_layer(full_2_dropout,120))
y_pred

<tf.Tensor 'add_5:0' shape=(?, 120) dtype=float32>

### Cross Entropy Loss and Training with Adam Optimizer

In [22]:
with tf.name_scope('Loss'):
    loss_op=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))
with tf.name_scope('Adam'):
    train_op=tf.train.AdamOptimizer().minimize(loss_op)    

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



### Accuracy

In [23]:
with tf.name_scope('Accuracy'):
    prediction=tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
    accuracy=tf.reduce_mean(tf.cast(prediction,tf.float32))

### Separating Training (9600) and Test Set (614)

In [24]:
images_train=images[:9600]
y_train=y[:9600]
images_test=images[9601:10214]
y_test=y[9601:10214]

### Graph and Summary

In [25]:
tf.summary.scalar("loss", loss_op)
tf.summary.scalar("accuracy", accuracy)
merged_summary_op = tf.summary.merge_all()
saver=tf.train.Saver()

### Running the Session (29 Epochs and Batch Size=32).Training accuracy varying from 34% to 60% . Maximium Test Accuracy 3.91%

In [26]:
with tf.Session() as sess:
    summary_writer = tf.summary.FileWriter('./output1', sess.graph)
    steps=300
    sess.run(tf.global_variables_initializer())
    for j in range(60):        
        for i in range(steps):         
            batch_x=augment(images_train[32*i:32*(i+1)])
            batch_y=(y_train[32*i:32*(i+1)])            
            print(sess.run([train_op,loss_op,accuracy],feed_dict={X1:batch_x,y_true:batch_y,hold_prob:0.7}))
            _, c, summary = sess.run([train_op, loss_op, merged_summary_op],feed_dict={X1: batch_x, y_true: batch_y,hold_prob:0.7})
            summary_writer.add_summary(summary,i)
        print('epoch',j,' loss,accuracy:',sess.run([loss_op,accuracy],feed_dict={X1:images_test,y_true:y_test,hold_prob:1.0}))    

[None, 4893.077, 0.0]
[None, 5538.9463, 0.0]
[None, 2756.3564, 0.0]
[None, 1177.677, 0.03125]
[None, 710.2772, 0.0]
[None, 534.45844, 0.0]
[None, 339.36865, 0.0]
[None, 267.16827, 0.0]
[None, 176.68404, 0.0]
[None, 121.619446, 0.0]
[None, 89.604004, 0.0]
[None, 80.23386, 0.0]
[None, 54.599987, 0.0]
[None, 37.656315, 0.03125]
[None, 35.99424, 0.03125]
[None, 32.784653, 0.0]
[None, 29.996014, 0.0]
[None, 26.214478, 0.0]
[None, 23.038979, 0.0]
[None, 22.364231, 0.0]
[None, 21.6862, 0.03125]
[None, 19.453835, 0.0]
[None, 22.5295, 0.0]
[None, 19.550217, 0.0]
[None, 14.732734, 0.0]
[None, 12.977764, 0.0]
[None, 16.172482, 0.0]
[None, 13.014595, 0.0]
[None, 14.356875, 0.0]
[None, 13.4366455, 0.0]
[None, 10.050138, 0.0]
[None, 9.2002125, 0.0]
[None, 11.31168, 0.0]
[None, 12.170913, 0.0]
[None, 10.846981, 0.03125]
[None, 9.783157, 0.0]
[None, 10.998762, 0.0]
[None, 8.413448, 0.0]
[None, 11.466643, 0.0]
[None, 8.440099, 0.0]
[None, 9.127086, 0.03125]
[None, 7.391143, 0.0]
[None, 7.992141, 0.0]
[

[None, 4.8448267, 0.0]
[None, 4.8625746, 0.0]
[None, 4.957235, 0.0]
[None, 4.794434, 0.0]
[None, 4.8732877, 0.0625]
[None, 4.9028835, 0.0625]
[None, 4.9960165, 0.0]
[None, 4.8935204, 0.03125]
[None, 4.910591, 0.0]
[None, 4.820286, 0.03125]
[None, 4.740945, 0.03125]
[None, 4.5644674, 0.0625]
[None, 4.925139, 0.0]
[None, 4.8513765, 0.0]
[None, 4.836323, 0.0]
[None, 4.8736224, 0.0]
[None, 4.8134503, 0.03125]
[None, 4.873025, 0.03125]
[None, 4.8560343, 0.03125]
[None, 4.920498, 0.0]
[None, 4.813448, 0.0]
[None, 4.8184814, 0.0]
[None, 4.9262304, 0.0]
[None, 4.757984, 0.03125]
[None, 4.949396, 0.0]
[None, 4.8480463, 0.0]
[None, 4.7791824, 0.0]
[None, 4.8697014, 0.0]
[None, 4.7538967, 0.0]
[None, 4.835713, 0.0]
[None, 4.918025, 0.0]
[None, 4.9256935, 0.0]
[None, 4.813431, 0.0625]
[None, 4.8206854, 0.03125]
[None, 4.833685, 0.0]
[None, 4.87888, 0.0]
[None, 4.8710423, 0.0]
[None, 4.912841, 0.0]
[None, 4.8574133, 0.0]
[None, 4.8318667, 0.0]
[None, 4.73635, 0.03125]
[None, 4.8236876, 0.0]
[None, 

[None, 4.793697, 0.0]
[None, 4.936821, 0.0]
[None, 4.7681236, 0.0]
[None, 4.8448277, 0.0]
[None, 4.88976, 0.0]
[None, 4.808738, 0.0]
[None, 4.8128014, 0.0]
[None, 4.745839, 0.0]
[None, 4.8411455, 0.0]
[None, 4.7409067, 0.0]
[None, 4.7862153, 0.03125]
[None, 4.7568703, 0.0]
[None, 4.777295, 0.0]
[None, 4.78603, 0.0]
[None, 4.7908583, 0.03125]
[None, 4.8130503, 0.0625]
[None, 4.9625177, 0.0]
[None, 4.7898216, 0.0]
[None, 4.679672, 0.03125]
[None, 4.789549, 0.0]
[None, 4.7558703, 0.0]
[None, 4.820404, 0.0]
[None, 4.725941, 0.03125]
[None, 4.7149725, 0.03125]
[None, 4.795143, 0.0]
[None, 4.756008, 0.0]
[None, 4.79675, 0.0]
[None, 4.7750835, 0.0]
[None, 4.7790575, 0.0]
[None, 4.7384977, 0.03125]
[None, 4.785278, 0.03125]
[None, 4.852909, 0.0]
[None, 4.8017693, 0.0]
[None, 4.814227, 0.0]
[None, 4.8316774, 0.0]
[None, 4.7926717, 0.0]
[None, 4.81645, 0.0]
[None, 4.80072, 0.0]
[None, 4.822976, 0.0]
[None, 4.7585917, 0.03125]
[None, 4.784852, 0.0]
[None, 4.701576, 0.03125]
[None, 4.825446, 0.0]


[None, 4.7659388, 0.03125]
[None, 4.79996, 0.0]
[None, 4.6526146, 0.03125]
[None, 4.8240824, 0.0]
[None, 4.7550898, 0.0]
[None, 4.7785664, 0.0]
[None, 4.7631483, 0.03125]
[None, 4.8136253, 0.03125]
[None, 4.7794676, 0.0]
[None, 4.8363075, 0.0]
[None, 4.6955776, 0.03125]
[None, 4.810751, 0.0]
[None, 4.7494183, 0.0]
[None, 4.707823, 0.03125]
[None, 4.7592015, 0.0625]
[None, 4.783829, 0.0]
[None, 4.7855487, 0.03125]
[None, 4.587652, 0.03125]
[None, 4.750783, 0.03125]
[None, 4.794779, 0.0]
[None, 4.7890635, 0.0]
[None, 4.74039, 0.0]
[None, 4.726595, 0.03125]
[None, 4.8149114, 0.03125]
[None, 4.7521844, 0.03125]
[None, 4.846357, 0.03125]
[None, 4.7176046, 0.0625]
[None, 4.764224, 0.03125]
[None, 4.7995768, 0.03125]
[None, 4.8951263, 0.0]
[None, 4.8971376, 0.0]
[None, 4.8524036, 0.0]
[None, 4.7954664, 0.0]
[None, 4.7801056, 0.0]
[None, 4.7623754, 0.03125]
[None, 4.8088927, 0.03125]
[None, 4.800997, 0.03125]
[None, 4.817976, 0.0]
[None, 4.7056093, 0.0]
[None, 4.805958, 0.0]
[None, 4.8831315, 

[None, 4.8277435, 0.03125]
[None, 4.948874, 0.0]
[None, 4.723162, 0.0]
[None, 4.8727674, 0.0]
[None, 4.7943025, 0.0]
[None, 4.8107624, 0.0]
[None, 4.7986727, 0.0]
[None, 4.8076763, 0.0]
[None, 4.7942724, 0.0]
[None, 4.7382326, 0.03125]
[None, 4.7498302, 0.03125]
[None, 4.7487707, 0.0]
[None, 4.7478223, 0.03125]
[None, 4.7399006, 0.0625]
[None, 4.6896486, 0.03125]
[None, 4.801978, 0.0]
[None, 4.726733, 0.0]
[None, 4.7782307, 0.0]
[None, 4.800525, 0.0]
[None, 4.766759, 0.0]
[None, 4.726036, 0.03125]
[None, 4.706549, 0.03125]
[None, 4.857826, 0.0]
[None, 4.841401, 0.0]
[None, 4.750334, 0.0]
[None, 4.817896, 0.0]
[None, 4.752015, 0.0]
[None, 4.801772, 0.0]
[None, 4.77003, 0.0]
[None, 4.869813, 0.0]
[None, 4.7341847, 0.03125]
[None, 4.7666054, 0.0]
[None, 4.7525086, 0.03125]
[None, 4.8080144, 0.03125]
[None, 4.928996, 0.0]
[None, 4.810506, 0.0]
[None, 4.847651, 0.03125]
[None, 4.715963, 0.0]
[None, 4.81734, 0.0]
[None, 4.687833, 0.03125]
[None, 4.695078, 0.0]
[None, 4.74724, 0.0]
[None, 4.7

...

[None, 3.289843, 0.34375]
[None, 3.0481215, 0.4375]
[None, 1.5661111, 0.71875]
[None, 3.2106352, 0.5625]
[None, 2.723541, 0.5625]
[None, 1.7203629, 0.75]
[None, 2.3188596, 0.65625]
[None, 2.4207134, 0.59375]
[None, 3.404683, 0.5]
[None, 2.7738423, 0.59375]
[None, 3.7418437, 0.25]
[None, 2.9212165, 0.46875]
[None, 2.5757785, 0.46875]
[None, 3.157635, 0.3125]
[None, 4.0274806, 0.21875]
[None, 3.8960593, 0.21875]
[None, 3.681214, 0.15625]
[None, 4.3306923, 0.1875]
[None, 3.549099, 0.34375]
[None, 3.8764486, 0.1875]
[None, 3.498678, 0.3125]
[None, 4.1989784, 0.28125]
[None, 3.975796, 0.25]
[None, 3.9182622, 0.25]
[None, 3.364281, 0.375]
[None, 3.1935563, 0.34375]
[None, 4.092602, 0.28125]
[None, 3.7458618, 0.25]
[None, 2.857925, 0.46875]
[None, 2.946238, 0.5]
[None, 2.6976888, 0.53125]
[None, 3.7067916, 0.375]
[None, 2.9681063, 0.5]
[None, 2.732819, 0.5]
[None, 3.1975496, 0.40625]
[None, 3.8603783, 0.25]
[None, 3.3582823, 0.3125]
[None, 3.7768443, 0.28125]
[None, 2.5952885, 0.4375]
[None, 

[None, 3.8485975, 0.21875]
[None, 4.050482, 0.28125]
[None, 3.7657757, 0.3125]
[None, 3.728179, 0.28125]
[None, 3.2867823, 0.3125]
[None, 4.739497, 0.25]
[None, 3.234366, 0.40625]
[None, 3.5614054, 0.3125]
[None, 3.1038601, 0.34375]
[None, 3.6677961, 0.28125]
[None, 3.122055, 0.34375]
[None, 3.4576077, 0.3125]
[None, 3.493835, 0.40625]
[None, 2.8317947, 0.5]
[None, 2.6308403, 0.5]
[None, 3.775958, 0.4375]
[None, 3.2195396, 0.40625]
[None, 4.4647517, 0.1875]
[None, 3.7211974, 0.21875]
[None, 3.9288044, 0.3125]
[None, 3.3733094, 0.25]
[None, 3.7385736, 0.375]
[None, 4.3900623, 0.1875]
[None, 2.4349823, 0.4375]
[None, 3.346662, 0.375]
[None, 4.0493665, 0.1875]
[None, 3.434927, 0.28125]
[None, 3.1211815, 0.375]
[None, 2.6093578, 0.5]
[None, 3.614396, 0.3125]
[None, 3.0123327, 0.46875]
[None, 1.7866068, 0.65625]
[None, 3.041355, 0.46875]
[None, 2.355909, 0.53125]
[None, 2.4740684, 0.625]
[None, 2.983336, 0.5]
[None, 2.0800881, 0.625]
[None, 2.4389906, 0.5]
[None, 2.6698875, 0.40625]
[None, 

[None, 2.3828664, 0.46875]
[None, 2.8690693, 0.4375]
[None, 3.164887, 0.40625]
[None, 3.2220564, 0.375]
[None, 3.729847, 0.28125]
[None, 3.3468137, 0.375]
[None, 2.75908, 0.46875]
[None, 4.2630663, 0.28125]
[None, 3.2108507, 0.4375]
[None, 2.5971642, 0.5]
[None, 2.7734833, 0.5]
[None, 3.4552608, 0.375]
[None, 2.4365578, 0.5625]
[None, 1.8921396, 0.65625]
[None, 3.0260768, 0.46875]
[None, 1.8848901, 0.65625]
[None, 2.001924, 0.59375]
[None, 2.3821802, 0.53125]
[None, 2.1848397, 0.65625]
[None, 2.544044, 0.53125]
[None, 1.4433397, 0.71875]
[None, 3.621281, 0.34375]
[None, 3.843774, 0.34375]
[None, 1.801033, 0.625]
[None, 2.8199453, 0.53125]
[None, 2.1876156, 0.625]
[None, 2.7129009, 0.4375]
[None, 1.9867678, 0.6875]
[None, 2.3713694, 0.53125]
[None, 2.1229892, 0.65625]
[None, 4.438736, 0.3125]
[None, 3.4587479, 0.40625]
[None, 2.5819836, 0.40625]
[None, 2.6834755, 0.5625]
[None, 4.0998306, 0.25]
[None, 3.553825, 0.40625]
[None, 3.2440434, 0.40625]
[None, 2.9730992, 0.4375]
[None, 3.40156

[None, 1.3129945, 0.84375]
[None, 2.3008676, 0.6875]
[None, 1.2972386, 0.8125]
[None, 1.7861316, 0.75]
[None, 1.5191646, 0.78125]
[None, 2.8979316, 0.5]
[None, 3.2280095, 0.4375]
[None, 2.8746681, 0.5]
[None, 2.7687736, 0.4375]
[None, 2.627382, 0.5]
[None, 2.848185, 0.375]
[None, 2.0962696, 0.625]
[None, 3.0843296, 0.59375]
[None, 3.438031, 0.34375]
[None, 2.348319, 0.5625]
[None, 2.331063, 0.53125]
[None, 2.2551913, 0.59375]
[None, 2.0445712, 0.71875]
[None, 2.8036544, 0.4375]
[None, 4.190153, 0.375]
[None, 3.5388978, 0.34375]
[None, 3.0633974, 0.4375]
[None, 2.8163137, 0.46875]
[None, 2.889457, 0.3125]
[None, 2.0311644, 0.5625]
[None, 3.0419762, 0.4375]
[None, 2.2448676, 0.59375]
[None, 1.7224824, 0.71875]


KeyboardInterrupt: 

In [ ]:
batch_x.shape

In [ ]:
train_data[0:50].shape

In [ ]:
train_data[:10]

In [ ]:
y_true.dtype

In [ ]:
X.shape